In [1]:
import sys, glob, os
SPARK_HOME="/usr/lib/spark-2.3.0-bin-hadoop2.7"
sys.path.append(SPARK_HOME + "/python")
sys.path.append(glob.glob(SPARK_HOME + "/python/lib/py4j*.zip")[0])
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sql = spark.sql
print(sc.uiWebUrl)

http://10.0.2.15:4040


In [3]:
type(spark)

pyspark.sql.session.SparkSession

In [8]:
movies = spark.read.options(header = True, inferSchema = True).csv("/data/movielens/movies.csv")
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:
type(movies)

pyspark.sql.dataframe.DataFrame

In [9]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [11]:
ratings = spark.read.options(header = True, inferSchema = True).csv("/data/movielens/ratings.csv")
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     16|   4.0|1217897793|
|     1|     24|   1.5|1217895807|
|     1|     32|   4.0|1217896246|
|     1|     47|   4.0|1217896556|
|     1|     50|   4.0|1217896523|
|     1|    110|   4.0|1217896150|
|     1|    150|   3.0|1217895940|
|     1|    161|   4.0|1217897864|
|     1|    165|   3.0|1217897135|
|     1|    204|   0.5|1217895786|
|     1|    223|   4.0|1217897795|
|     1|    256|   0.5|1217895764|
|     1|    260|   4.5|1217895864|
|     1|    261|   1.5|1217895750|
|     1|    277|   0.5|1217895772|
|     1|    296|   4.0|1217896125|
|     1|    318|   4.0|1217895860|
|     1|    349|   4.5|1217897058|
|     1|    356|   3.0|1217896231|
|     1|    377|   2.5|1217896373|
+------+-------+------+----------+
only showing top 20 rows



In [12]:
movies.count()

10329

In [13]:
ratings.count()

105339

In [16]:
movies.filter("title like 'Godfather%'").show(10, False)

+-------+-------------------------------+----------------------------+
|movieId|title                          |genres                      |
+-------+-------------------------------+----------------------------+
|858    |Godfather, The (1972)          |Crime|Drama                 |
|1221   |Godfather: Part II, The (1974) |Crime|Drama                 |
|2023   |Godfather: Part III, The (1990)|Crime|Drama|Mystery|Thriller|
+-------+-------------------------------+----------------------------+



In [18]:
import pyspark.sql.functions as F

In [19]:
ratings.filter("movieId = 858").agg(F.avg("rating"), F.count("rating")).show()

+-----------------+-------------+
|      avg(rating)|count(rating)|
+-----------------+-------------+
|4.392857142857143|          210|
+-----------------+-------------+



In [27]:
agg = (ratings.groupBy("movieId")
.agg(F.avg("rating").alias("avg_rating")
            , F.count("rating").alias("rating_count"))
.filter("rating_count >= 100")
.orderBy(F.desc("avg_rating"))
 .limit(10)
 .join(movies, on = ["movieId"])
 .select("movieId", "title", "avg_rating")
)

agg.show()

+-------+--------------------+------------------+
|movieId|               title|        avg_rating|
+-------+--------------------+------------------+
|     50|Usual Suspects, T...| 4.328947368421052|
|    318|Shawshank Redempt...| 4.454545454545454|
|    527|Schindler's List ...| 4.296370967741935|
|    608|        Fargo (1996)|4.2711442786069655|
|    858|Godfather, The (1...| 4.392857142857143|
|   1136|Monty Python and ...|4.3019480519480515|
|   1193|One Flew Over the...|4.2727272727272725|
|   1213|   Goodfellas (1990)|4.2592592592592595|
|   1221|Godfather: Part I...| 4.260714285714286|
|   2571|  Matrix, The (1999)| 4.264367816091954|
+-------+--------------------+------------------+



In [30]:
agg.coalesce(1).write.format("csv").mode("overwrite").save("/data/top10movies")